# Fine-tuning DistilBERT to Build a Fake News Classifier

In this notebook, we’ll build a binary classification model to detect fake news using **DistilBERT**, a streamlined version of **BERT** — one of the foundational large language models (LLMs). We’ll fine-tune a pretrained DistilBERT model on our fake news dataset, evaluate how well it generalizes to *unseen* news articles, and then test it on a very *real* story from the front page of *The New York Times*.

You can check out the baseline model we'll be using on Hugging Face [right here](https://huggingface.co/distilbert/distilbert-base-uncased). Before we dive in, here's a quick rundown of what is to come:

## The Pipeline:
- Setup and data import
- Load the pretrained model from Hugging Face
- Prepare the dataset for PyTorch
- Fine-tune the model
- Generate predictions on unseen (test) data
- Measure classification performance
- Save the model for future use
- Use fine-tuned DistilBERT to classify a *real* news story
- Build the `classify_article` function

## Setup and data import

First up: Start by verifying GPU access and installing any necessary packages.

In [ ]:
# Check the type of GPU available in the Colab environment
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fd72e57d-9c25-8bef-45be-a392f8da07b0)


In [ ]:
# Install (or upgrade) gdown to enable downloading datasets from Google Drive
# You might need to restart the runtime after this line
!pip install --upgrade gdown

In [ ]:
# Download the Fake News dataset from Google Drive using gdown
import gdown
gdown.download('https://drive.google.com/uc?id=11usJt37HhpYIVglRY8aD8-EUomsF-1tL')

Downloading...
From (original): https://drive.google.com/uc?id=11usJt37HhpYIVglRY8aD8-EUomsF-1tL
From (redirected): https://drive.google.com/uc?id=11usJt37HhpYIVglRY8aD8-EUomsF-1tL&confirm=t&uuid=bb83976d-1c8c-4e9b-8db8-2494d2ef8f1e
To: /content/fake_news.csv
100%|██████████| 117M/117M [00:01<00:00, 103MB/s]


'fake_news.csv'

Now we'll import pandas and create the `fake_news` DataFrame. Then we'll craft it to combine the text fields and simplify the columns for modeling.

In [ ]:
# Import pandas and load the dataset into a DataFrame
import pandas as pd
fake_news = pd.read_csv('fake_news.csv',index_col=0)

# Combine the title and text into a single 'content' column
fake_news['content'] = fake_news['title'] + '. ' + fake_news['text']

# Drop the columns we no longer neeed
fake_news = fake_news.drop(columns=['title','text','subject','date'])
fake_news

,fake,content
0,1,Donald Trump Sends Out Embarrassing New Year’...
1,1,Drunk Bragging Trump Staffer Started Russian ...
2,1,Sheriff David Clarke Becomes An Internet Joke...
3,1,Trump Is So Obsessed He Even Has Obama’s Name...
4,1,Pope Francis Just Called Out Donald Trump Dur...
...,...,...
44893,0,'Fully committed' NATO backs new U.S. approach...
44894,0,LexisNexis withdrew two products from Chinese ...
44895,0,Minsk cultural hub becomes haven from authorit...
44896,0,Vatican upbeat on possibility of Pope Francis ...


In [ ]:
# Quickly load the first article in the DataFrame for reference
fake_news.content[0]

' Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing. Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this

We know have a streamlined dataset with one key column: `content` which contains the full article body. This is what we’ll feed into our BERT tokenizer. Worth noting: This is a big dataset, consisting of nearly 45K news articles.

## Load the pretrained model from HuggingFace

In [ ]:
# Install transformers
! pip install transformers

In [ ]:
# First we will make two imports: 1) The model for sequence classification and 2) The corresponding tokenizer
# Worth noting: Sequence classification is really text classification because it means sequences of words
from transformers import DistilBertForSequenceClassification
from transformers import DistilBertTokenizerFast

# And now create two objects
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Prepare the Dataset for PyTorch

The dataset needs to be turned into a PyTorch datset so that the model will be able to efficiently process it in batches during training and evaluation.

In [ ]:
# Set X (the input features) and y (the target labels)
# So: X is a series of documents and y is equal to labels, 1's and 0's or fake or not fake
X = fake_news.content
y = fake_news.fake

Split the dataset in three: the training set (20%), the validation set (10%), and the test set (70%). The validation set is used to evaluate the model's performance during the training process.

**Note**: The training set is small because it's incredibly time consuming to train a model.

In [ ]:
# Step 1: Import test_train split from scikit-learn
from sklearn.model_selection import train_test_split

# Step 2a: Split the datset into a small training set (20%) and a large remainder (80%)
X_train, X_rem, y_train, y_rem = train_test_split(X,y,test_size=0.8,random_state=0)

# Step 2b: Split the remainder into validation (10%) and test (70%)
# The validation set helps us evaluate and tune the model during training
# The test set is used only once at the end to evaluate final model performance
X_valid,X_test,y_valid,y_test = train_test_split(X_rem,y_rem,test_size=7.0/8.0,random_state=1)

In [ ]:
# Check the length of the training set
len(X_train)

8979

In [ ]:
# And check the length of the validation set
len(X_valid)

4489

In [ ]:
# And finally the test set, which should be 7x the validation set
len(X_test)

31430

Next, we need to tokenize our input and turn it into encodings -- numeric IDs that represent each word (or subword) in the model's vocabulary.

This is essential because models like BERT don’t understand raw text -- they process sequences of token IDs. We’ll start with a small example to see how this works, and then apply it to the entire dataset.

In [ ]:
# Here's an example of two sentences
sentences=['I want to be good at natural language processing','I also want to work as little as I can']

In [ ]:
# Now call the tokenizer we loaded earlier and ask it to tokenize the sentences
# Truncation=True and padding=True because BERT expects all input sequences to be the same length
# Truncation shortens longer sentences, and padding fills shorter ones
# Notice in the output below "I" equals "101."
tokenizer(sentences,truncation=True,padding=True)

{'input_ids': [[101, 1045, 2215, 2000, 2022, 2204, 2012, 3019, 2653, 6364, 102, 0], [101, 1045, 2036, 2215, 2000, 2147, 2004, 2210, 2004, 1045, 2064, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

Before feeding our text into the model, we need to tokenize it using the BERT tokenizer. This will turn each sentence into a sequence of token IDs based on the model’s vocabulary. These token IDs are what the model actually "sees" and processes.

In [ ]:
# Now to turn the test, training and validation sets into encodings
# As you see, we are turning the Series into lists
# Note: This could take a while to produce
train_encodings = tokenizer(X_train.values.tolist(),truncation=True,padding=True)
val_encodings = tokenizer(X_valid.values.tolist(),truncation=True,padding=True)
test_encodings = tokenizer(X_test.values.tolist(),truncation=True,padding=True)

In [ ]:
# Check the first encoded training example
enc = train_encodings[0]

# Use .ids to view the actual token ID sequence for that document
enc.ids

[101,
 3422,
 1024,
 9659,
 2080,
 7861,
 8237,
 8180,
 8583,
 2624,
 5348,
 3664,
 2040,
 2056,
 8398,
 1521,
 1055,
 1523,
 4288,
 1524,
 5984,
 13641,
 2015,
 1024,
 1523,
 1045,
 2123,
 1521,
 1056,
 2156,
 2111,
 5996,
 1524,
 1012,
 2292,
 1055,
 3246,
 2023,
 4854,
 24247,
 3664,
 2987,
 1056,
 4652,
 3265,
 17843,
 2013,
 8020,
 2000,
 2393,
 1996,
 2111,
 1997,
 2624,
 5348,
 2040,
 2089,
 2025,
 5993,
 2007,
 2014,
 5223,
 3993,
 4491,
 2006,
 2256,
 2343,
 1010,
 2040,
 2003,
 14457,
 2058,
 8848,
 2000,
 2131,
 1996,
 2393,
 2000,
 1996,
 5694,
 1997,
 1996,
 7064,
 1012,
 2003,
 3087,
 2842,
 5305,
 1998,
 5458,
 1997,
 4994,
 2023,
 3664,
 1059,
 14014,
 2055,
 2673,
 2637,
 2003,
 2025,
 2725,
 2005,
 2014,
 1029,
 2624,
 5348,
 3664,
 11425,
 9805,
 4115,
 8096,
 1010,
 2040,
 2038,
 4188,
 2000,
 5589,
 1999,
 2151,
 10768,
 2863,
 6295,
 4953,
 4681,
 2000,
 5984,
 7043,
 1999,
 1996,
 10530,
 1997,
 7064,
 3814,
 1010,
 2038,
 14678,
 5496,
 1996,
 8398,
 3447,
 1997

## Fine-tune the model

Here's the high-level rundown of the block of text below:

This code defines a custom PyTorch Dataset class so that the model can consume the tokenized inputs and associated labels -- X and y -- in a way that PyTorch understands. It turns them into **PyTorch tensors**, which are like NumPy arrays with superpowers for training on GPUs.

In [ ]:
# First, import PyTorch so that we can build and manage datasets and tensors
import torch

# Define a custom dataset class so that PyTorch knows how to feed data into our model
# This class takes in tokenized inputs and labels and formats them in PyTorch-friendly chunks
class MyDataset(torch.utils.data.Dataset):

    # Initialize the class with the encodings (input Ids, attention masks and so on) and labels (0 or 1)
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # This is how PyTorch will grab one item from the dataset during training
    def __getitem__(self, idx):
        # Grab the idx-th example, turn each part into a tensor
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        # Add the label (0 or 1) for this specific example
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # This just tells PyTorch how many items are in the dataset
    def __len__(self):
        return len(self.labels)

# Now we use that custom class to create datasets for training, validation and testing
# These are the versions that PyTorch will feed into the model
train_dataset = MyDataset(train_encodings, y_train.values.tolist())
val_dataset = MyDataset(val_encodings, y_valid.values.tolist())
test_dataset = MyDataset(test_encodings, y_test.values.tolist())

In other words: This function turns our dataset into a format that PyTorch can understand and train on -- without it, the model has no idea how to use our inputs or labels.

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=1,              # Total number of training epochs, or how many times to scan the dataaset
    per_device_train_batch_size=16,  # How many observations to see in the training set before picking the parameters for the next step
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    report_to='none'
)

trainer = Trainer(
    model=model,                         # The instantiated 🤗 Transformers model to be trained
    args=training_args,                  # Training arguments, defined above
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=val_dataset             # Evaluation dataset
)

trainer.train()

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


Step,Training Loss
10,0.715120
20,0.691559
30,0.677961
40,0.669438
50,0.646653
60,0.585248
70,0.506737
80,0.371066
90,0.225778
100,0.123548


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=562, training_loss=0.10812165284487071, metrics={'train_runtime': 487.7502, 'train_samples_per_second': 18.409, 'train_steps_per_second': 1.152, 'total_flos': 1189424772532224.0, 'train_loss': 0.10812165284487071, 'epoch': 1.0})

## Generate predictions on unseen (test) data

Now that the model is trained, we're using it to generate predicitons on the test dataset. This is done using the `Trainer` object's `.predict()` method.

In [ ]:
# Use the fine-tuned model to generate predictions on the test set
# This helps evaluate generalizations to unseen data
# Note: This is a really slow process
test_predictions = trainer.predict(test_dataset)

In [ ]:
# Now check the test_predictions
# Each element in the test set has two scores about how confident we are in a prediction of not fake or fake
test_predictions

PredictionOutput(predictions=array([[-3.961303 ,  3.7852027],
       [-4.0353537,  3.828378 ],
       [ 3.8530095, -4.6025367],
       ...,
       [-4.0599866,  3.8108275],
       [ 3.8802266, -4.6466837],
       [-3.5559657,  3.3099816]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 0, 1]), metrics={'test_loss': 0.004497068002820015, 'test_runtime': 491.9439, 'test_samples_per_second': 63.889, 'test_steps_per_second': 1.0})

In [ ]:
# Need to translate the scores into probabilites by taking the soft max
# So, this output gives us the probabilites, each pair adds up to 1!
# And save as probas
probas = torch.softmax(torch.Tensor(test_predictions.predictions),dim=1)
probas

tensor([[4.3206e-04, 9.9957e-01],
        [3.8429e-04, 9.9962e-01],
        [9.9979e-01, 2.1267e-04],
        ...,
        [3.8158e-04, 9.9962e-01],
        [9.9980e-01, 1.9803e-04],
        [1.0416e-03, 9.9896e-01]])

In [ ]:
# And finally to get the binary test predictions
# So, want to turn those above values into 0s or 1s with .argmax()
# And axis=1 means, of course, we are selecting the max row across each row
# And finally save as binary test predictions
binary_test_predictions = probas.argmax(axis=1)
binary_test_predictions

tensor([1, 1, 0,  ..., 1, 0, 1])

## Measure classification performance

Next we'll use scikit-learn to evaluate how well our fine-tuned model performs on the test set.

In [ ]:
# Start by importing the classification report from scikit-learn
# and ise the actual labels -- y_test -- and the binary_test_predictions we just calculated
from sklearn.metrics import classification_report
print(classification_report(y_test, binary_test_predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15006
           1       1.00      1.00      1.00     16424

    accuracy                           1.00     31430
   macro avg       1.00      1.00      1.00     31430
weighted avg       1.00      1.00      1.00     31430



100%! That looks amazing. Let's go more in depth...

In [ ]:
# Now to find the errors
# First, need to import numpy and make it an np.array
import numpy as np
np.array(binary_test_predictions)

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
# And now to compare with the actual test values
y_test.values != np.array(binary_test_predictions)

array([False, False, False, ..., False, False, False])

In [ ]:
# It's pretty much all False, so let's check the sum
(y_test.values != np.array(binary_test_predictions)).sum()

np.int64(22)

Wow. So, 27 errors out of more than 31,000 predictions. And now to confirm with the confusion matrix.

In [ ]:
# Import the confusion matrix from scikit-learn and run it
# Woeth noting: Confusion matrix layout...
# [True Negatives, False Positive],
# [False Negatives, True Positives]
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, binary_test_predictions)

array([[14987,    19],
       [    3, 16421]])

**Final takeaway**: And there it is. The model is, indeed, wildly accurate. Out of 31,000 predictions, only 21 stories were misclassified as fake and just 6 were misclassified as real. Of course, it's important to note that this is **a very data-hungry model** as we saw through the fine-tuning process.

## Save the model for future use

First, let's save it in the Colab environment because otherwise it will disappear when he close the runtime.

In [ ]:
# Create a new directory
# And then save the model and the tokenizer
model_path = 'fake-news-distilbert'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('fake-news-distilbert/tokenizer_config.json',
 'fake-news-distilbert/tokenizer.json')

Finally, copy it into your Google Drive so that these files don't disappear!

In [ ]:
# And now to make it visible
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Import shell util
import shutil
shutil.copytree(model_path, '/content/drive/MyDrive/' + model_path, dirs_exist_ok=True)


'/content/drive/MyDrive/fake-news-distilbert'

The fine-tuned `DistilBERT` model and tokenizer are now safely stored in Google Drive. Ready for reuse, transfer, or deployment. Speaking of which...

## Use the model to classify real news story

Now let's see how our fine-tuned `DistilBERT` does when asked to classify a real news story taken from the homepage of *The New York Times*. To begin, load the model and tokenizer that was just saved in Google Drive.

In [ ]:
# Make the imports
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch

model_path = '/content/drive/MyDrive/fake-news-distilbert'

# Load themodel and tokenizer from the saved local directory
# This tells Hugging Face to look locally on the Google Drive, not on the internet
model = DistilBertForSequenceClassification.from_pretrained(model_path, local_files_only=True)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path, local_files_only=True)

Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

Next up, it's time to provide the text from a sample story. Let's try this one from the NYT: "[Trump Is Poised to Accept a Luxury 747 From Qatar for Use as Air Force One](https://www.nytimes.com/2025/05/11/us/politics/trump-qatar-jet-gift-air-force-one.html)."

**Important note**: The model truncates anything over its max input size -- usually 512 tokens or about 350-400 words.

In [ ]:
# Copy the story here
story = """The Trump administration plans to accept a luxury Boeing 747-8 plane as a donation from the Qatari royal family that will be upgraded to serve as Air Force One, which would make it one of the biggest foreign gifts ever received by the U.S. government, several American officials with knowledge of the matter said.
The plane would then be donated to President Trump’s presidential library when he leaves office, two senior officials said. Such a gift raises the possibility that Mr. Trump would have use of the plane even after his presidency ends.
Mr. Trump confirmed the fact that he anticipates receiving the plane in a post on social media on Sunday evening, after a day of controversy in which even some Republicans privately questioned the wisdom of the plan. Mr. Trump suggested that Democrats were “losers” for questioning the ethics of the move.
“So the fact that the Defense Department is getting a GIFT, FREE OF CHARGE, of a 747 aircraft to replace the 40 year old Air Force One, temporarily, in a very public and transparent transaction, so bothers the Crooked Democrats that they insist we pay, TOP DOLLAR, for the plane,” Mr. Trump wrote. “Anybody can do that! The Dems are World Class Losers!!!”

While a Qatari official described the proposal as still under discussion and the White House said that gifts it accepted would be done in full compliance with the law, Democratic lawmakers and good government groups expressed outrage over the substantial ethical issues the plan presented. They cited the intersection of Mr. Trump’s official duties with his business interests in the Middle East, the immense value of the lavishly appointed plane and the assumption that Mr. Trump would have use of it after leaving office. Sold new, a commercial Boeing 747-8 costs in the range of $400 million.
“Even in a presidency defined by grift, this move is shocking,” said Robert Weissman, a co-president of Public Citizen, a consumer advocacy organization. “It makes clear that U.S. foreign policy under Donald Trump is up for sale.”
Mr. Trump’s own private plane, known as “Trump Force One,” is an older 757 jet that first flew in the early 1990s and was then used by the Microsoft co-founder Paul Allen. Mr. Trump bought it in 2011. The Qatari jet, if Mr. Trump continued flying it after leaving office, would give him a substantially newer plane for his own use.
ABC News reported Sunday morning that the gift of the plane was to be announced in the coming days as Mr. Trump made the first extended foreign trip of his presidency to three nations in the Middle East, including Qatar. The plan would fulfill the president’s desire for a new Air Force One after repeated delays involving a government contract to Boeing for two new jets to serve that purpose.
In a statement, the White House press secretary, Karoline Leavitt, said: “Any gift given by a foreign government is always accepted in full compliance with all applicable laws. President Trump’s administration is committed to full transparency.”

An agreement for the government to accept the luxury aircraft and ultimately pass it along to Mr. Trump’s library would be the clearest example yet of how he has further intertwined his personal and presidential business in his second term. While Mr. Trump faced criticism during his first term for the way his properties collected money from the government, the last four months have included a flurry of deals around a cryptocurrency firm that has erased centuries-old presidential norms.

While two senior U.S. officials said the plane would be accepted as a gift for use by the government, Ali Al-Ansari, a spokesman for the Qatari government, said that reports of the plane being offered “during the upcoming visit of President Trump are inaccurate.”
He added, “The possible transfer of an aircraft for temporary use as Air Force One is currently under consideration between Qatar’s Ministry of Defense and the U.S. Department of Defense, but the matter remains under review by the respective legal departments, and no decision has been made.”
A White House official echoed that the plane would not be presented or accepted this week. The Justice Department and the Defense Department did not comment.
Mr. Trump toured the Qatari-owned 747, which is just over a decade old, while it was parked at Palm Beach International Airport in February. The New York Times reported then that the jet was being considered as a possible new Air Force One.
One senior U.S. government official said that the Defense Department had determined it could accept the plane. Two people familiar with the language of an analysis conducted by the office of the White House counsel, David Warrington, and the attorney general, Pam Bondi, whose past work as a lobbyist included Qatar as a client, said they had determined it would fall within the law for Mr. Trump’s library to receive the plane. The two people spoke on the condition of anonymity to discuss plans that have yet to be publicly announced.
The plane that would be donated by Qatar is expected to be retrofitted by a military contractor called L3Harris, in Texas, and that work can begin once the government approves how the plane is being acquired, one of the senior officials said. Assuming the transfer of the plane goes forward, it is expected to be finished being equipped with military capabilities by the end of the year, the official added, allowing Mr. Trump to use it while in office.
The status of the expected contract with L3Harris was not immediately clear.
A Defense Department official said on Sunday that the Air Force has not yet reached any agreement on a contract to refurbish the Qatari 747 to make the security upgrades and modifications necessary for an AF1, and the Air Force could not legally do so until it actually took ownership of the plane.
Assuming that happens, the official said, it would still take an extended period of time to complete the contract and, more important, to make the actual modifications to match a full Air Force One upgrade.
“We’re talking years, not months,” the Defense Department official said, speaking on condition of anonymity to discuss sensitive details about a future Air Force One.
Advertisement
Others with knowledge of the project have suggested that the Qatari plane could be a more modified version of an Air Force One, and therefore could be built out faster.
Since shortly after his election, Mr. Trump has been frustrated with the budget overruns and delays with the two new Air Force One planes that Boeing is on contract to deliver to the federal government. The two current Air Force Ones are more than 30 years old and need frequent servicing, sometimes taking months.
Mr. Trump renegotiated the contract for the planes in his first term, and has wanted to fly on them while in office now.
Throughout the White House, officials have repeatedly complained about the Boeing delays.
Mr. Trump asked Elon Musk, the billionaire chief executive of SpaceX, to find ways to accelerate the planes’ delivery. Mr. Musk had insisted that one of the planes could be delivered in a year’s time.
One option that company officials and Mr. Musk had discussed was lowering security clearance requirements for those working on parts of the plane that do not relate to the most secure parts of an Air Force One jet. It is unclear where those talks led.
Advertisement
Some involved in developing a plane that could survive the fallout of a nuclear attack as well as avoid some missile attacks have said that Mr. Musk’s projected time frame is unrealistic.
But the Trump administration considered acquiring a Qatari-owned plane as a potential alternative to waiting for the two other Boeing jets, and a possible motivator for the company to move faster.
The model that the government is using for addressing the ethical issues raised by the donation, one of the officials said, is the one followed by President Ronald Reagan’s presidential library when it received the Air Force One he had flown on after it was retired from use. But at the time, Mr. Reagan did not use the plane to fly around. It was set up in the museum portion of his library.
Another person with knowledge of the effort to acquire the plane said that the Qataris had initially discussed donating it immediately to the Trump library, and Mr. Trump would then use it while in office. But government lawyers said that would violate the emoluments clause of the Constitution, the person said, which prohibits federal officials from accepting financial benefits from foreign governments without congressional approval.
The plan, if it goes through, is almost certain to revive those concerns. In 2021, the Supreme Court declined to adjudicate similar questions. That January, it vacated lower court rulings in cases accusing Mr. Trump of violating the clause in his first term, as he was no longer in office.
A spokesman for the government watchdog group Citizens for Responsibility and Ethics in Washington, which sued him over what it described as violations of the emoluments clause during his first term, said the president’s acceptance of the plane would raise a range of ethical issues far beyond whether it violated the Constitution, in part because he may continue using it after leaving office.
“It’s hard to see it as a coincidence when Trump’s company just announced a new golf resort in Qatar, reportedly partnered with a company owned by the country’s government, and will soon be meeting with senior Qatari officials in a Middle East trip that also features meetings with heads of state of two other countries he has property developments in,” the spokesman, Jordan Libowitz, said in a statement.
“At this point, it’s impossible to tell the difference between decisions being made by the White House for the good of the country and for the good of the Trump Organization,” he added.
The reported deal also drew fierce criticism from Democrats.
Senator Adam B. Schiff of California, who as a House member led the first impeachment case against Mr. Trump, assailed the plan. “The corruption is brazen,” Mr. Schiff wrote on social media.
Representative Jamie Raskin of Maryland criticized the gift on social media, saying the president must first seek Congress’s consent.
“The Constitution is perfectly clear: no present ‘of any kind whatever’ from a foreign state without Congressional permission,” he wrote.
"""

In [ ]:
# Check the output of story
story

'The Trump administration plans to accept a luxury Boeing 747-8 plane as a donation from the Qatari royal family that will be upgraded to serve as Air Force One, which would make it one of the biggest foreign gifts ever received by the U.S. government, several American officials with knowledge of the matter said.\nThe plane would then be donated to President Trump’s presidential library when he leaves office, two senior officials said. Such a gift raises the possibility that Mr. Trump would have use of the plane even after his presidency ends.\nMr. Trump confirmed the fact that he anticipates receiving the plane in a post on social media on Sunday evening, after a day of controversy in which even some Republicans privately questioned the wisdom of the plan. Mr. Trump suggested that Democrats were “losers” for questioning the ethics of the move.\n“So the fact that the Defense Department is getting a GIFT, FREE OF CHARGE, of a 747 aircraft to replace the 40 year old Air Force One, tempor

In [ ]:
# And now let's tokenize it with truncation
inputs = tokenizer(story, truncation=True, padding=True, return_tensors='pt')

In [ ]:
# So, this shows us the output that the model will be predicting on
inputs

{'input_ids': tensor([[  101,  1996,  8398,  3447,  3488,  2000,  5138,  1037,  9542, 10321,
         25374,  1011,  1022,  4946,  2004,  1037, 13445,  2013,  1996, 12577,
          2072,  2548,  2155,  2008,  2097,  2022,  9725,  2000,  3710,  2004,
          2250,  2486,  2028,  1010,  2029,  2052,  2191,  2009,  2028,  1997,
          1996,  5221,  3097,  9604,  2412,  2363,  2011,  1996,  1057,  1012,
          1055,  1012,  2231,  1010,  2195,  2137,  4584,  2007,  3716,  1997,
          1996,  3043,  2056,  1012,  1996,  4946,  2052,  2059,  2022,  6955,
          2000,  2343,  8398,  1521,  1055,  4883,  3075,  2043,  2002,  3727,
          2436,  1010,  2048,  3026,  4584,  2056,  1012,  2107,  1037,  5592,
         13275,  1996,  6061,  2008,  2720,  1012,  8398,  2052,  2031,  2224,
          1997,  1996,  4946,  2130,  2044,  2010,  8798,  4515,  1012,  2720,
          1012,  8398,  4484,  1996,  2755,  2008,  2002,  3424,  6895, 17585,
          2015,  4909,  1996,  4946,  

In [ ]:
# Run the model on our tokenized input
with torch.no_grad():
    output = model(**inputs)

In [ ]:
# Extract the raw scores, or logits, and choose the higher one as the prediction
logits = output.logits
prediction = torch.argmax(logits, dim=1).item()

In [ ]:
# Optional: View the raw output scores (logits) if you want to see model confidence
# This is commented for now
#logits

In [ ]:
# Optional: Check the prediction class as 0 -- Real -- or 1 -- Fake
# This is also commented for now
# prediction

And now time for **the grand reveal...**

In [ ]:
# Translate the numeric prediction into a readable label
label = 'Fake' if prediction == 1 else 'Real'
print(f'📰 Prediction: {label} News')

📰 Prediction: Fake News


How wild. But now let's see how confident the model is that this story is, indeed, fake news.

In [ ]:
# Turn the logits to probabilities using softmax
probas = torch.softmax(logits, dim=1)

# Extract the confidence score for the predicted class
confidence = probas[0][prediction].item()

# Display result with confidence
print(f'Prediction: {label} News (Confidence: {confidence:.2%})')


Prediction: Fake News (Confidence: 99.69%)


**Final Takeaway**: There is it. With 99.69% confidence, the model -- fine-tuned `DistilBERT` -- believes that this is a fake news. It is, of course, *real* coming directly from the front page of *The New York Times*. And yet, the model that only made 27 errors on a dataset of more than 31,000 articles, saw this one and said: *No way. This can't be true*. This math-based machine learning model, trained to detect fake news, thinks this one is too outrageous to be true.

Honestly? Hard to blame it.

## Build the `classify_article` function

Below you'll find the `classify_article` function, which allows you to run any article through the model that we just fine-tuned and tested.

In [ ]:
# Function: classify a news using the fine-tuned DistilBERT model
def classify_article(text, model, tokenizer):
    """
    Classify a single news article using a fine-tuned DistilBERT model.

    Parameters:
    - text (str): Raw article text to classify
    - model: Pre-loaded fine-tuned DistilBERT model
    - tokenizer: Matching tokenizer

    Returns:
    - label (str): 'Fake" or "Real'
    - confidence (float): Confidence score for the predicted label
    """
    # Tokenize the input with truncation and padding
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='pt')

    # Run the model on the input
    with torch.no_grad():
        output = model(**inputs)

    # Extract logits and convert to probabilities
    logits = output.logits
    probas = torch.softmax(logits, dim=1)

    # Get predicted class and confidence
    prediction = torch.argmax(probas, dim=1).item()
    confidence = probas[0][prediction].item()

    # Map prediction to label
    label = 'Fake' if prediction == 1 else 'Real'

    return label, confidence

And here's where you can drop in your story.

In [ ]:
# Paste your full article here (use triple quotes for multi-paragraph text)
story_text = """
INSERT STORY TEXT HERE
"""

# Run the classifier
label, confidence = classify_article(story_text, model, tokenizer)

# Print the result
print(f'Prediction: {label} News (Confidence: {confidence:.2%})')


Prediction: Fake News (Confidence: 99.87%)
